#load related library

In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame


from ipywidgets import widgets 
from ipywidgets import *  
from IPython.display import display,clear_output
#https://github.com/jupyter-widgets/ipywidgets/issues/134 about ouput
from ipywidgets import Layout
import folium
import datetime
from datetime import date
from dateutil import rrule

from traitlets import directional_link

# install rtree
# https://github.com/kjordahl/SciPy-Tutorial-2015/issues/1
# http://jspeis.com/installing-rtree-on-mac-os-x/

plt.style.use('ggplot')

def loadJson(filename):
	import json
	f = open(filename, "r", encoding = "utf-8")
	output = json.load(f)
	f.close()
	return output

# load geo information for analyzed data

column: title,	geometry

In [2]:

# load all geo information
names=['title', 'lon', 'lat']
geodata = pd.read_csv("title_long_lat.csv", names=names)
geometry = [Point(xy) for xy in zip(geodata['lon'], geodata['lat'])]

# load event set (with _)
event_set = set([one for one in loadJson("eventList.json")])
# load people set
people_set = set([one for one in loadJson("personList.json")])

print (len([one for one in geodata['title'] if one in event_set]))

print (len([one for one in geodata['title'] if one in people_set]))

event_geodata = geodata[geodata['title'].isin(event_set)]
event_geometry = [Point(xy) for xy in zip(event_geodata['lon'], event_geodata['lat'])]
people_geodata = geodata[geodata['title'].isin(people_set)]
people_geometry = [Point(xy) for xy in zip(people_geodata['lon'], people_geodata['lat'])]
other_geodata = geodata[~geodata['title'].isin(event_set)]
other_geodata = other_geodata[~geodata['title'].isin(people_set)]
other_geometry = [Point(xy) for xy in zip(other_geodata['lon'], other_geodata['lat'])]

# event_gdf = GeoDataFrame(event_geodata, geometry=event_geometry)
# people_gdf = GeoDataFrame(people_geodata, geometry=people_geometry)
# other_gdf = GeoDataFrame(other_geodata, geometry=other_geometry)

sub_gdf = {}
sub_gdf["People"] = GeoDataFrame(people_geodata, geometry=people_geometry)[['title','geometry']] #people_gdf
sub_gdf["Events"] = GeoDataFrame(event_geodata, geometry=event_geometry)[['title','geometry']] # event_gdf
sub_gdf["Others"] = GeoDataFrame(other_geodata, geometry=other_geometry)[['title','geometry']] # other_gdf

# sub_geodata = {}
# sub_geodata["People"]=people_geodata
# sub_geodata["Events"]=event_geodata
# sub_geodata["Others"]=other_geodata

print(len(geodata))
print (len(event_geodata))
print (len(people_geodata))
print (len(other_geodata))

7754
1911


/Users/yqh/anaconda/lib/python3.4/site-packages/pandas/core/frame.py:1825: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


956916
7754
1911
947251


In [3]:
#how the data sub_geodata["People"] looks like
sub_gdf["People"].head()

,title,geometry
16,Alfred_Nobel,POINT (18.01928611111111 59.35681111111111)
19,A._A._Milne,POINT (0.107 51.09)
36,Abraham,POINT (35.110726 31.524744)
39,Ahmad_Shah_Durrani,POINT (65.70694444444446 31.61944444444445)
40,Aga_Khan_III,POINT (32.878722 24.088254)


In [3]:
#load score information

pos_score_OL = loadJson("score_file/pos_score_OL.json")
neg_score_OL = loadJson("score_file/neg_score_OL.json")
pos_neg_score_OL = loadJson("score_file/pos_neg_score_OL.json")

pos_score_MPQA = loadJson("score_file/pos_score_MPQA.json")
neg_score_MPQA = loadJson("score_file/neg_score_MPQA.json")
pos_neg_score_MPQA = loadJson("score_file/pos_neg_score_MPQA.json")

pos_score_LIWC = loadJson("score_file/pos_score_LIWC.json")
neg_score_LIWC = loadJson("score_file/neg_score_LIWC.json")
pos_neg_score_LIWC = loadJson("score_file/pos_neg_score_LIWC.json")

scores = {}
scores["OL"] = {}
scores["MPQA"] = {}
scores["LIWC"] = {}
    
scores["OL"]["positive"] = pos_score_OL
scores["OL"]["negative"] = neg_score_OL
scores["OL"]["positive+negative"] = pos_neg_score_OL
scores["MPQA"]["positive"] = pos_score_MPQA
scores["MPQA"]["negative"] = neg_score_MPQA
scores["MPQA"]["positive+negative"] = pos_neg_score_MPQA
scores["LIWC"]["positive"] = pos_score_LIWC
scores["LIWC"]["negative"] = neg_score_LIWC
scores["LIWC"]["positive+negative"] = pos_neg_score_LIWC

# get border info for world map

column: country, geometry, continent

In [5]:
# get borders of countries 
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
print ("structure of 'world':")
print (world.head())
# pick only country name and geometry column
country_border = world[['name','geometry', 'continent']]
# rename column name to 'country','geometry'
country_border = country_border.rename(columns={'name':'country'})
print ("structure of 'country_border':")
print (country_border.head())

#cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

# build continent->country dict for dropdown

#continent_list = world.continent.unique()
continent_list = ["all"] + list(set(world['continent']))
#continent_list.append("all")

continent_country_dict = {}
for continent in continent_list:
    continent_country_dict[continent] = ["all"] + list(world[world['continent']== continent]['name'])
    #continent_country_dict[continent].append("all")
    
print ("continent list for dropdown:")
print (continent_list)
for one in ["People","Events","Others"]:
    sub_gdf[one].crs = country_border.crs
#print (conti_country[1])
#area_dict = {}

structure of 'world':
    pop_est      continent                  name iso_a3  gdp_md_est  \
0  28400000           Asia           Afghanistan    AFG       22270   
1  12799293         Africa                Angola    AGO      110300   
2   3639453         Europe               Albania    ALB       21810   
3   4798491           Asia  United Arab Emirates    ARE      184300   
4  40913584  South America             Argentina    ARG      573900   

                                            geometry  
0  POLYGON ((61.21081709172574 35.65007233330923,...  
1  (POLYGON ((16.32652835456705 -5.87747039146621...  
2  POLYGON ((20.59024743010491 41.85540416113361,...  
3  POLYGON ((51.57951867046327 24.24549713795111,...  
4  (POLYGON ((-65.50000000000003 -55.199999999999...  
structure of 'country_border':
                country                                           geometry  \
0           Afghanistan  POLYGON ((61.21081709172574 35.65007233330923,...   
1                Angola  (POLYGON 

# Current Version


In [26]:


# preparing a container for side panel
sp_container = widgets.VBox(layout=Layout(
    border='solid 1px',
    width='30%'
)
)
style = {'description_width':'50px'}
my_layout = Layout(border='solid 1px')

# for sentiment lexicon
# lexicon container for lexicons
lexicon_container = widgets.VBox(layout=my_layout)
# add title
label_lexicon = widgets.Label(value="sentiment lexicon")
# preparing a container to put in radio buttons
radio_button_lexicon = widgets.RadioButtons(
    options=['OL', 'MPQA', 'LIWC'],
    #description='sentiment lexicon',
    #style=style,
    disabled=False
)
# put text and button into lexicon container
lexicon_container.children = [label_lexicon,radio_button_lexicon]


# for target group
# container for groups
group_container = widgets.VBox(layout=my_layout)
label_group = widgets.Label(value="target group")
# checkboxes container
cb_container = widgets.VBox(layout=Layout(
)) 
# preparing a container to put in created checkbox
checkboxes = []  
# create checkbox
checkboxes.append(widgets.Checkbox(description = 'People', value=False))
checkboxes.append(widgets.Checkbox(description = 'Events', value=False))
checkboxes.append(widgets.Checkbox(description = 'Others', value=False))
# put check box into checkboxes container
cb_container.children=[i for i in checkboxes]
#display(cb_container)
# # add a new container to control the arrangement
# temp_container = widgets.HBox()
# temp = widgets.Label(description='choose target group')
# temp_container.children=[cb_container,temp]
group_container.children=[label_group,cb_container]


# for target sentiment, pos | neg | pos+neg
# container for sentiment
sentiment_container = widgets.VBox(layout=my_layout)
label_sentiment = widgets.Label(value="target sentiment")
# checkboxes container
st_container = widgets.VBox(
    #description='target sentiment'
)
radio_button_sentiment = widgets.RadioButtons(
    options=['positive', 'negative', 'positive+negative'],
    #description='sentiment',
    #style=style,
    disabled=False
)
sentiment_container.children=[label_sentiment,radio_button_sentiment]


# for area filter
#container for area
area_container = widgets.VBox(layout=my_layout)
label_area = widgets.Label(value="target area")
#dropdown_container = widgets.HBox()
#dropdown
dropdown_continent = widgets.Dropdown(
    options = continent_list,
    value = 'all',
    description='continent:',
    layout=Layout(width='250px'),
    disabled=False,
)
    
dropdown_country = widgets.Dropdown(
    #options = country_list,
    description='country:',
    #options = ["all"],
    #value='all',
    layout=Layout(width='250px'),
    
    disabled = False,
)

def transform(case):
    return continent_country_dict[case]

directional_link((dropdown_continent,'value'),(dropdown_country,'options'),transform)


#l = traitlets.link((dropdown_continent,'value'),(dropdown_country,'value'))
#dropdown_container.children = [dropdown_continent,dropdown_country]
area_container.children=[label_area,dropdown_continent,dropdown_country]


# container for slider
slider_container = widgets.VBox(layout=my_layout)
# add a time slider: SelectionRangeSlider
dates = [day for day in rrule.rrule(rrule.MONTHLY, dtstart=date(2001,1,1), until=date.today())]
options = [(i.strftime('%b%Y'),i) for i in dates]

slider = widgets.SelectionRangeSlider(
    options=options,
    index=(0,192),
    #description='time range',
    layout=Layout(width='250px',height='40px'),
    #,description_width='200px'
    #description_width:'200px'
    style = style,
    disabled=False
)
label_slider = widgets.Label(value="time range")
slider_container.children=[label_slider,slider]


#for button
update_container = widgets.VBox(layout=my_layout)
#add button that updates the graph based on the checkboxes
button = widgets.Button(description="Update the graph")
update_container.children=[button]


sp_container.children=[lexicon_container, group_container, area_container, slider_container, button]


# for map
map_container = widgets.VBox(layout=Layout(width='80%',height='100%',border='solid 4px'))
# preparing the plot 
myMap = folium.Map(location=[45.5236, -122.6750], zoom_start=1)
map_out = widgets.Output()
with map_out:
    display(myMap)
map_container.children = [map_out]
    
# container for all
all_container = widgets.HBox(layout=Layout(border='solid 4px')
)
all_container.children=[map_container,sp_container]
display(all_container)
#print (dir(myMap))

def on_button_clicked(b): 
#     # delete map and create a new one
    #import rtree
    myMap = folium.Map(location=[45.5236, -122.6750], zoom_start=1)
    # get dropdown info
    node_continent = dropdown_continent.value
    node_country = dropdown_country.value
    print (node_continent,node_country)
    cc_e = 0
    for c in cb_container.children:
        
        if c.value:
            group = c.description
            # area filter
            if not node_continent == "all":
                if not node_country == "all":
                    target_data = gpd.sjoin(sub_gdf[group],country_border,how="inner",op='intersects')
                    target_data = target_data[target_data['country']==node_country]
                else:
                    target_data = gpd.sjoin(sub_gdf[group],country_border,how="inner",op='intersects')
                    target_data = target_data[target_data['continent']==node_continent]
            else:
                target_data = sub_gdf[group]
            #print (target_data)
            print ("totally %d nodes will be shown"%(len(target_data)))
            
            
            for each in target_data.iterrows():              
                #print (node_continent,node_country)
                #print ("finished")
                #break
                #[each[1]['lat'],each[1]['lon']]
                title = each[1]['title']
                title = title.replace("_", " ")
                pos_score_dict = scores[radio_button_lexicon.value]["positive"]
                if title in pos_score_dict:
                    #print ("there")
                    cc_e += 1
#                     if cc_e >= 20:
#                         break
                    if cc_e % 500 == 0:
                        print (cc_e)
                    
                    #deal with color
                    pos_score = pos_score_dict[title]
                    #print (pos_score)
                    neg_score = scores[radio_button_lexicon.value]["negative"][title]
                    #print (neg_score)
                    
#                     if pos_score + neg_score >= 1000:
#                         continue
                    if (pos_score + neg_score) == 0:
                        node_color = 'yellow'
                    else:
                        # formular
                        # ff * pos_score/total_score
                        # 
                        pos_color = str(hex(int(255 * pos_score/(pos_score+neg_score))))[2:]
                        neg_color = str(hex(int(255 * neg_score/(pos_score+neg_score))))[2:]
                        #print (pos_color,neg_color)
                        node_color = "#"+neg_color+"00"+pos_color
#                     pos_color = (1000-pos_score) * 255/1000
                    
#                     pos_color = hex(int(pos_color))
                    
#                     pos_color = str(pos_color)[2:]
#                     #print ("pos_color:"+pos_color)
#                     #print (neg_score)
#                     neg_color = int((1000-neg_score) * 255/1000)
#                     #print (neg_color)
#                     neg_color = str(hex(neg_color))[2:]
#                     #print ("neg_color:"+neg_color)
                
                    #print (node_color)
                    #print (title+node_color)
                    #break
                    #print ((scores[radio_button_lexicon.value]["positive+negative"][title]))
                    popup = folium.Popup(title,parse_html=True)
                    folium.CircleMarker([each[1]['geometry'].y,each[1]['geometry'].x],
                                  radius=(scores[radio_button_lexicon.value]["positive+negative"][title])/20,
                                  #radius = 5,     
                                  popup=popup,
                                  fill = True,
                                  color=node_color,
                                  fill_color=node_color,
                                 ).add_to(myMap)
                    
        else:
            # delete
            print ("here")
    print (cc_e)
    with map_out:
        clear_output(wait=True)
        display(myMap)
    print ("finish")

button.on_click(on_button_clicked)  



HBox(children=(VBox(children=(Output(),), layout=Layout(border='solid 4px', height='100%', width='80%')), VBox(children=(VBox(children=(Label(value='sentiment lexicon'), RadioButtons(options=('OL', 'MPQA', 'LIWC'), value='OL')), layout=Layout(border='solid 1px')), VBox(children=(Label(value='target group'), VBox(children=(Checkbox(value=False, description='People'), Checkbox(value=False, description='Events'), Checkbox(value=False, description='Others')))), layout=Layout(border='solid 1px')), VBox(children=(Label(value='target area'), Dropdown(description='continent:', layout=Layout(width='250px'), options=('all', 'Seven seas (open ocean)', 'South America', 'Asia', 'Europe', 'Antarctica', 'Oceania', 'North America', 'Africa'), value='all'), Dropdown(description='country:', layout=Layout(width='250px'), options=('all',), value='all')), layout=Layout(border='solid 1px')), VBox(children=(Label(value='time range'), SelectionRangeSlider(index=(0, 192), layout=Layout(height='40px', width='250px'), options=(('Jan2001', datetime.datetime(2001, 1, 1, 0, 0)), ('Feb2001', datetime.datetime(2001, 2, 1, 0, 0)), ('Mar2001', datetime.datetime(2001, 3, 1, 0, 0)), ('Apr2001', datetime.datetime(2001, 4, 1, 0, 0)), ('May2001', datetime.datetime(2001, 5, 1, 0, 0)), ('Jun2001', datetime.datetime(2001, 6, 1, 0, 0)), ('Jul2001', datetime.datetime(2001, 7, 1, 0, 0)), ('Aug2001', datetime.datetime(2001, 8, 1, 0, 0)), ('Sep2001', datetime.datetime(2001, 9, 1, 0, 0)), ('Oct2001', datetime.datetime(2001, 10, 1, 0, 0)), ('Nov2001', datetime.datetime(2001, 11, 1, 0, 0)), ('Dec2001', datetime.datetime(2001, 12, 1, 0, 0)), ('Jan2002', datetime.datetime(2002, 1, 1, 0, 0)), ('Feb2002', datetime.datetime(2002, 2, 1, 0, 0)), ('Mar2002', datetime.datetime(2002, 3, 1, 0, 0)), ('Apr2002', datetime.datetime(2002, 4, 1, 0, 0)), ('May2002', datetime.datetime(2002, 5, 1, 0, 0)), ('Jun2002', datetime.datetime(2002, 6, 1, 0, 0)), ('Jul2002', datetime.datetime(2002, 7, 1, 0, 0)), ('Aug2002', datetime.datetime(2002, 8, 1, 0, 0)), ('Sep2002', datetime.datetime(2002, 9, 1, 0, 0)), ('Oct2002', datetime.datetime(2002, 10, 1, 0, 0)), ('Nov2002', datetime.datetime(2002, 11, 1, 0, 0)), ('Dec2002', datetime.datetime(2002, 12, 1, 0, 0)), ('Jan2003', datetime.datetime(2003, 1, 1, 0, 0)), ('Feb2003', datetime.datetime(2003, 2, 1, 0, 0)), ('Mar2003', datetime.datetime(2003, 3, 1, 0, 0)), ('Apr2003', datetime.datetime(2003, 4, 1, 0, 0)), ('May2003', datetime.datetime(2003, 5, 1, 0, 0)), ('Jun2003', datetime.datetime(2003, 6, 1, 0, 0)), ('Jul2003', datetime.datetime(2003, 7, 1, 0, 0)), ('Aug2003', datetime.datetime(2003, 8, 1, 0, 0)), ('Sep2003', datetime.datetime(2003, 9, 1, 0, 0)), ('Oct2003', datetime.datetime(2003, 10, 1, 0, 0)), ('Nov2003', datetime.datetime(2003, 11, 1, 0, 0)), ('Dec2003', datetime.datetime(2003, 12, 1, 0, 0)), ('Jan2004', datetime.datetime(2004, 1, 1, 0, 0)), ('Feb2004', datetime.datetime(2004, 2, 1, 0, 0)), ('Mar2004', datetime.datetime(2004, 3, 1, 0, 0)), ('Apr2004', datetime.datetime(2004, 4, 1, 0, 0)), ('May2004', datetime.datetime(2004, 5, 1, 0, 0)), ('Jun2004', datetime.datetime(2004, 6, 1, 0, 0)), ('Jul2004', datetime.datetime(2004, 7, 1, 0, 0)), ('Aug2004', datetime.datetime(2004, 8, 1, 0, 0)), ('Sep2004', datetime.datetime(2004, 9, 1, 0, 0)), ('Oct2004', datetime.datetime(2004, 10, 1, 0, 0)), ('Nov2004', datetime.datetime(2004, 11, 1, 0, 0)), ('Dec2004', datetime.datetime(2004, 12, 1, 0, 0)), ('Jan2005', datetime.datetime(2005, 1, 1, 0, 0)), ('Feb2005', datetime.datetime(2005, 2, 1, 0, 0)), ('Mar2005', datetime.datetime(2005, 3, 1, 0, 0)), ('Apr2005', datetime.datetime(2005, 4, 1, 0, 0)), ('May2005', datetime.datetime(2005, 5, 1, 0, 0)), ('Jun2005', datetime.datetime(2005, 6, 1, 0, 0)), ('Jul2005', datetime.datetime(2005, 7, 1, 0, 0)), ('Aug2005', datetime.datetime(2005, 8, 1, 0, 0)), ('Sep2005', datetime.datetime(2005, 9, 1, 0, 0)), ('Oct2005', datetime.datetime(2005, 10, 1, 0, 0)), ('Nov2005', datetime.datetime(2005, 11, 1, 0, 0)), ('Dec2005', datetime

Europe all
totally 558 nodes will be shown
500
here
here
541
finish
Europe all
here


In [19]:
#help (folium.CircleMarker)
help(folium)
#print (folium._version)
    

Help on package folium:

NAME
    folium - # -*- coding: utf-8 -*-

PACKAGE CONTENTS
    _version
    features
    folium
    map
    plugins (package)
    utilities

CLASSES
    builtins.object
        branca.element.Element
            branca.element.Figure
                branca.element.Div
            branca.element.Html
            branca.element.IFrame
            branca.element.Link
                branca.element.CssLink
                branca.element.JavascriptLink
            branca.element.MacroElement
                branca.colormap.ColorMap
                    branca.colormap.LinearColormap
                    branca.colormap.StepColormap
                folium.features.ClickForMarker
                folium.features.DivIcon
                folium.features.LatLngPopup
                folium.map.FitBounds
                folium.map.Icon
                    folium.features.CustomIcon
                folium.map.LayerControl
                folium.map.Marker
                    

In [5]:
import pandas as pd
names=['title', 'lon', 'lat']
data_1 = pd.read_csv("title_long_lat_2.csv", names=names)
data_2 = pd.read_csv("title_long_lat_3.csv", names=names)

data_3 = pd.concat([data_2,data_1],ignore_index=True)
data_4 = data_3.drop_duplicates(subset='title')

data_4.to_csv('title_long_lat_4.csv',columns=names,header=False,index=False,encoding='utf-8')
#data_4.to_csv('title_long_lat_title.csv',columns=['title'],header=False,index=False,encoding='utf-8')
#data_4.to_csv('title_long_lat_long_lat.csv',columns=['lon','lat'],header=False,index=False,encoding='utf-8')
# add " " automatically if there is comma inside title


In [47]:
data_8 = pd.concat([data_1,data_2],join='outer')
data_8

,title,lon,lat
0,Algeria,3.216667,36.700000
1,Andorra,1.516667,42.500000
2,Aruba,-70.033333,12.516667
3,Angola,13.333333,-8.833333
0,Alabama,-86.700000,32.700000
1,Algeria,2.000000,28.000000
2,Apollo_8,-165.016667,8.133333
3,Apollo_11,-169.150000,13.316667
4,Andorra,1.500000,42.500000


In [3]:
data_4 = data_1.append(data_2)
data_4

,title,lon,lat
0,Algeria,3.216667,36.700000
1,Andorra,1.516667,42.500000
2,Aruba,-70.033333,12.516667
3,Angola,13.333333,-8.833333
0,Alabama,-86.700000,32.700000
1,Algeria,2.000000,28.000000
2,Apollo_8,-165.016667,8.133333
3,Apollo_11,-169.150000,13.316667
4,Andorra,1.500000,42.500000


In [5]:
data_5 = pd.concat([data_1,data_2],ignore_index=True)
data_5

,title,lon,lat
0,Algeria,3.216667,36.700000
1,Andorra,1.516667,42.500000
2,Aruba,-70.033333,12.516667
3,Angola,13.333333,-8.833333
4,Alabama,-86.700000,32.700000
5,Algeria,2.000000,28.000000
6,Apollo_8,-165.016667,8.133333
7,Apollo_11,-169.150000,13.316667
8,Andorra,1.500000,42.500000


In [8]:
data_6 = pd.merge(data_1,data_2,how='outer',on=['title'])
data_6

,title,lon_x,lat_x,lon_y,lat_y
0,Algeria,3.216667,36.700000,2.000000,28.000000
1,Andorra,1.516667,42.500000,1.500000,42.500000
2,Aruba,-70.033333,12.516667,NaN,NaN
3,Angola,13.333333,-8.833333,NaN,NaN
4,Alabama,NaN,NaN,-86.700000,32.700000
5,Apollo_8,NaN,NaN,-165.016667,8.133333
6,Apollo_11,NaN,NaN,-169.150000,13.316667


In [36]:
data_7 = data_5.duplicated()
data_7.head()
data_5
# data_7 = data_1.join(data_2,on='title',how='outer',)
# data_7

,title,lon,lat
0,Algeria,3.216667,36.700000
1,Andorra,1.516667,42.500000
2,Aruba,-70.033333,12.516667
3,Angola,13.333333,-8.833333
4,Alabama,-86.700000,32.700000
6,Apollo_8,-165.016667,8.133333
7,Apollo_11,-169.150000,13.316667


In [23]:
data_2

,title,lon,lat
0,Alabama,-86.700000,32.700000
1,Algeria,2.000000,28.000000
2,Apollo_8,-165.016667,8.133333
3,Apollo_11,-169.150000,13.316667
4,Andorra,1.500000,42.500000
